In [1]:
%%bash
git add nlp100_80.ipynb

In [30]:
%%bash
git commit -a -m "snapshot"

[master 482fa9c] snapshot
 2 files changed, 127 insertions(+), 1 deletion(-)


****0 コーパスの整形 ****


文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．

- トークンの先頭と末尾に出現する次の文字を削除: .,!?;:()[]'"
- 空文字列となったトークンは削除

以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ

In [4]:
import bz2

In [7]:
import codecs

In [5]:
fname_input = "nlp100data/enwiki-20150112-400-r100-10576.txt.bz2"

In [6]:
fname_output = "nlp100data/corpus80.txt"

- bz2.open の引数"rt"の"t"はtextを表す。省略するとbyte型になってしまう。
- chunk.strip()は引数を指定しないとwhitespaceを除去

In [33]:
with bz2.open(fname_input  , 'rt') as rf , codecs.open(fname_output , "w" , "utf-8") as wf:
    for i , l in enumerate(rf):
        if i > -1:
            tokens = []
            for chunk in l.split(' '):
                token = chunk.strip().strip('.,!?;:()[]\'"')
                if len(token) > 0:
                    tokens.append(token)
                    
            processed = " ".join(tokens) + "\n"
            wf.write(processed)

                


In [34]:
 " ".join(tokens) + "\n"

'\n'

#### 複合語からなる国名への対処
英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．

インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである．

https://gist.github.com/kalinchernev/486393efcca01623b18d

In [ ]:
%%bash
cd nlp100data

wget https://gist.githubusercontent.com/kalinchernev/486393efcca01623b18d/raw/daa24c9fea66afb7d68f8d69f0c4b8eeb9406e83/countries

In [50]:
with codecs.open("nlp100data/countries"
                 , "r" , "utf-8") as rf:
    country_names = [l.strip('\n') for l in rf]

    

In [51]:
country_names

['Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua & Deps',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Central African Rep',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Congo {Democratic Rep}',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'East Timor',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Honduras',
 'Hungary',
 'Iceland',
 'India',
 'Indones

In [54]:
len(country_names)

196

In [52]:
%%bash
git commit -a -m "doing 81"

[master 4cbc3d5] doing 81
 1 file changed, 322 insertions(+), 11 deletions(-)


In [61]:
compound_countries = dict()
for c in country_names:
    c_split = c.split(" ")
    n = len(c_split)
    if n > 1:
        compound_countries[c] = "_".join(c_split)
        

In [68]:
import re

In [81]:
fname_output81 = "nlp100data/output81.txt"

In [83]:
with codecs.open(fname_output , "r" , "utf-8") as rf , codecs.open(fname_output81 , "w" , "utf-8") as wf:
    for i, l in enumerate(rf):
        for k ,v in compound_countries.items():
            l = l.replace(k , v)
        #print(l)
        wf.write(l)
        #if i > 10:
         #   break

In [85]:
%%bash
git commit -a -m "finished 81"

[master 7ff2723] finished 81
 1 file changed, 24 insertions(+), 38 deletions(-)


##### sandbox

In [84]:
with codecs.open(fname_output81 , "r" , "utf-8") as rf:
    for i, l in enumerate(rf):
        print(l)
        if i > 3:
            break

Anarchism



Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions but that several authors have defined as more specific institutions based on non-hierarchical free associations Anarchism holds the state to be undesirable unnecessary or harmful While anti-statism is central anarchism entails opposing authority or hierarchical organisation in the conduct of human relations including but not limited to the state system

As a subtle and anti-dogmatic philosophy anarchism draws on many currents of thought and strategy Anarchism does not offer a fixed body of doctrine from a single particular world view instead fluxing and flowing as a philosophy There are many types and traditions of anarchism not all of which are mutually exclusive Anarchist schools of thought can differ fundamentally supporting anything from extreme individualism to complete collectivism Strains of anarchism have often been divided into the categorie

In [67]:
for k  , v in compound_countries.items():
    print(k , v)

Sierra Leone Sierra_Leone
Central African Rep Central_African_Rep
Ivory Coast Ivory_Coast
Saudi Arabia Saudi_Arabia
South Sudan South_Sudan
Trinidad & Tobago Trinidad_&_Tobago
Dominican Republic Dominican_Republic
Cape Verde Cape_Verde
Czech Republic Czech_Republic
Korea North Korea_North
New Zealand New_Zealand
Saint Vincent & the Grenadines Saint_Vincent_&_the_Grenadines
Costa Rica Costa_Rica
Russian Federation Russian_Federation
South Africa South_Africa
Sri Lanka Sri_Lanka
Papua New Guinea Papua_New_Guinea
St Kitts & Nevis St_Kitts_&_Nevis
Myanmar, {Burma} Myanmar,_{Burma}
Antigua & Deps Antigua_&_Deps
United States United_States
San Marino San_Marino
United Kingdom United_Kingdom
Congo {Democratic Rep} Congo_{Democratic_Rep}
East Timor East_Timor
Solomon Islands Solomon_Islands
Ireland {Republic} Ireland_{Republic}
Marshall Islands Marshall_Islands
St Lucia St_Lucia
Equatorial Guinea Equatorial_Guinea
United Arab Emirates United_Arab_Emirates
El Salvador El_Salvador
Vatican City V

#### 文脈の抽出
81で作成したコーパス中に出現するすべての単語$t$に関して，単語$t$と文脈語$c$のペアをタブ区切り形式ですべて書き出せ．ただし，文脈語の定義は次の通りとする．

- ある単語$t$の前後$d$単語を文脈語$c$として抽出する（ただし，文脈語に単語$t$そのものは含まない）
- 単語$t$を選ぶ度に，文脈幅$d$は{1,2,3,4,5}の範囲でランダムに決める．





In [92]:
import random

In [98]:
random.randint(1,5)

5

In [124]:
list82 = list()
with codecs.open(fname_output81 , "r" , "utf-8") as rf:
    for i, l in enumerate(rf):
        v = l.strip().split(' ')
        
        
        for j , w in enumerate(v):
            d = random.randint(1 , 5)
            vcontext = [v[k] for k in set(range(j-d , j + d + 1)) & set(range(len(v))) - set([j]) ]
            
            list82.append(w + "\t" + " ".join(vcontext))
        if i > 99999999999:
            break

In [125]:
len(list82)

11901002

In [128]:
[print(l) for i , l in enumerate(list82) if i < 10] 

Anarchism	
	
Anarchism	is
is	Anarchism a political
a	is political
political	Anarchism is a philosophy that advocates stateless societies
philosophy	Anarchism is a political that advocates stateless societies often
that	Anarchism is a political philosophy advocates stateless societies often defined
advocates	is a political philosophy that stateless societies often defined as
stateless	societies advocates


[None, None, None, None, None, None, None, None, None, None]

In [87]:
with codecs.open(fname_output81 , "r" , "utf-8") as rf:
    for i, l in enumerate(rf):
        print(l)
        if i > 3:
            break

Anarchism



Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions but that several authors have defined as more specific institutions based on non-hierarchical free associations Anarchism holds the state to be undesirable unnecessary or harmful While anti-statism is central anarchism entails opposing authority or hierarchical organisation in the conduct of human relations including but not limited to the state system

As a subtle and anti-dogmatic philosophy anarchism draws on many currents of thought and strategy Anarchism does not offer a fixed body of doctrine from a single particular world view instead fluxing and flowing as a philosophy There are many types and traditions of anarchism not all of which are mutually exclusive Anarchist schools of thought can differ fundamentally supporting anything from extreme individualism to complete collectivism Strains of anarchism have often been divided into the categorie

In [88]:

l.strip??